In [1]:
import os
import torch
import torch.nn as nn
from torch.nn import CrossEntropyLoss # это плохо, но иначе не читается конфиг из словаря
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import random_split
import torchvision
import torchvision.transforms as transforms

from my_models import AlexNet

In [2]:
def load_data(data_dir='./data/FashionMNIST'):
    transform = transforms.Compose([
        transforms.ToTensor()
    ])
    
    trainset = torchvision.datasets.FashionMNIST(
        root=data_dir, train=True, download=True, transform=transform)

    testset = torchvision.datasets.FashionMNIST(
        root=data_dir, train=False, download=True, transform=transform)

    return trainset, testset

In [3]:
def test_accuracy(net, device = "cuda:0"):
    trainset, testset = load_data()

    testloader = torch.utils.data.DataLoader(
        testset, batch_size=4, shuffle=False)


    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)

            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return correct / total

In [4]:
with open("./data/best_trial_dir.txt", "r") as file:
    best_trained_model_path = file.readline()[:-2]
    config = eval(file.readline())

In [5]:
gpus_per_trial=1

best_trained_model = AlexNet(config=config)  
device = "cpu"
if torch.cuda.is_available():
    device = "cuda:0"
    if gpus_per_trial > 1:
        best_trained_model = nn.DataParallel(best_trained_model)
best_trained_model.to(device=device)

model_state, optimizer_state = torch.load(os.path.join(
        best_trained_model_path, "checkpoint"))
best_trained_model.load_state_dict(model_state)


<All keys matched successfully>

In [6]:
gpus_per_trial=1
device = "cpu"
if torch.cuda.is_available():
    device = "cuda:0"
    if gpus_per_trial > 1:
        best_trained_model = nn.DataParallel(best_trained_model)

test_acc = test_accuracy(best_trained_model, device=device)   

In [7]:
 print("Best trial test set accuracy: {}".format(test_acc))

Best trial test set accuracy: 0.8539
